<a href="https://colab.research.google.com/github/JalesBussinguer/Remote_Sensing_Studies/blob/master/Desafio_Worcap_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação das bibliotecas básicas

import pandas as pd
import numpy as np
import io
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# Importação das bibliotecas e módulos de machine learning

from sklearn import ensemble, preprocessing, tree
from sklearn.metrics import auc, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from yellowbrick.classifier import ConfusionMatrix, ROCAUC
from yellowbrick.model_selection import learning_curve

In [ ]:
# Importando os dados de treino

df = "https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/dados_verdade_terrestre.csv"

dados = pd.read_csv(df)

dados.head()

,id,b1,b2,b3,b4,b5,b6,b7,b8,b9,pred_minus_obs_H_b1,pred_minus_obs_H_b2,pred_minus_obs_H_b3,pred_minus_obs_H_b4,pred_minus_obs_H_b5,pred_minus_obs_H_b6,pred_minus_obs_H_b7,pred_minus_obs_H_b8,pred_minus_obs_H_b9,pred_minus_obs_S_b1,pred_minus_obs_S_b2,pred_minus_obs_S_b3,pred_minus_obs_S_b4,pred_minus_obs_S_b5,pred_minus_obs_S_b6,pred_minus_obs_S_b7,pred_minus_obs_S_b8,pred_minus_obs_S_b9,label
0,454,54,28,51,93,56,100,80,24,55,64.88,26.03,50.29,6.74,-30.83,-39.83,0.47,5.50,0.47,-24.52,-0.83,-3.99,-23.22,-0.94,-4.30,-23.22,-1.69,-4.55,s
1,457,54,29,51,88,51,91,95,24,55,56.28,20.62,42.67,10.16,-26.12,-32.05,-21.37,5.39,-0.30,-19.03,-0.54,-4.09,-22.24,-1.01,-4.93,-17.81,-0.53,-2.98,s
2,261,59,30,52,90,54,93,80,26,58,56.58,21.37,45.45,12.77,-28.17,-30.94,-1.30,3.64,-2.27,-21.04,-1.86,-5.61,-27.73,-1.15,-5.74,-21.81,-2.39,-5.07,s
3,8,53,27,49,95,49,92,63,25,54,66.97,24.43,49.28,8.08,-22.53,-28.25,19.78,3.75,0.92,-25.65,-2.09,-5.95,-39.27,-2.13,-8.73,-30.73,-2.42,-5.58,s
4,478,58,40,65,100,58,100,106,26,57,51.79,8.74,28.06,-1.00,-33.12,-41.04,-32.17,3.48,-2.08,-18.31,-0.20,-4.85,-21.15,-1.00,-4.84,-17.00,-0.91,-3.38,d


O conjunto de dados **`treino.csv`** será utilizado para construir os modelos de aprendizagem de máquina. Este apresenta a "verdade em terra" para cada classe por meio da coluna `label`.

In [ ]:
# Importação dos dados de validação

dados_validacao = pd.read_csv("https://raw.githubusercontent.com/JalesBussinguer/Remote_Sensing_Studies/master/data/worcap_2020/dados_validacao.csv")
dados_validacao.head(5)

dados_val_sid = dados_validacao.drop(columns="id")

O conjunto **`teste.csv`** será utilizado para verificar a capacidade de generalização do modelo criado.

## Dicionário de variáveis:

* `id`: Identificação única da linha, alusiva a um pixel único das imagens;
* `b1`: Banda do verde corresponde ao mês de setembro de 2010;
* `b2`: Banda do vermelho corresponde ao mês de setembro de 2010;
* `b3`: Banda do infravermelho próximo corresponde ao mês de setembro de 2010;
* `b4`: Banda do verde corresponde ao mês de Março de 2011;
* `b5`: Banda do vermelho corresponde ao mês de Março de 2011;
* `b6`: Banda do infravermelho próximo corresponde ao mês de Março de 2011;
* `b7`: Banda do verde corresponde ao mês de Maio de 2011;
* `b8`: Banda do vermelho corresponde ao mês de Maio de 2011;
* `b9`: Banda do infravermelho próximo corresponde ao mês de Maio de 2011;
* `pred_minus_obs_S_b1` até `pred_minus_obs_S_b9`: Valores espectrais previstos (Com base na interpolação espacial) subtraídos dos valores espectrais reais para a classe "s";
* `pred_minus_obs_H_b1` até `pred_minus_obs_H_b9`: Valores espectrais previstos (Com base na interpolação espacial) subtraídos dos valores espectrais reais para a classe "h".

## Rótulos:

* `S` = *Sugi* (Floresta composta predominantemente pela espécie [*Cryptomeria japonica*](https://pt.wikipedia.org/wiki/Cryptomeria_japonica))
* `H` = *Hinoki* (Floresta composta predominantemente pela espécie [*Chamaecyparis obtusa*](https://en.wikipedia.org/wiki/Chamaecyparis_obtusa))
* `D` = *Mixed Deciduous* (Formação florestal composta por diversas espécies deciduais ou caducifólias);
* `O` = *Other* (Feição não-florestal)

Por se tratar de um problema de classificação de múltiplas classes, os resultados obtidos com a utlização de Regressão Logística e KNN podem não ser acurados. Dessa forma, serão utilizados somente os algoritmos de Árvore de Decisão e Random Forest Classification.

# Análise Exploratória

Nesta primeira fase, é interessante verificar algumas características do dataset, para certificar que tudo está ok para a implementação do modelo de aprendizado de máquina.

## 1. Conferindo se existem dados faltando

In [ ]:
# Expressão que conta os dados que faltam pelo tamanho do dataset, retornando a porcentagem faltante de dados

dados.isnull().sum()/dados.shape[0]

No conjunto de dados de treino, não há dados faltantes. Portanto, podemos presseguir na análise.

In [ ]:
dados_validacao.isnull().sum()/dados_validacao.shape[0]

No conjunto de dados de teste também não existem dados faltantes. Portanto, prosseguimos para uma análise dos tipos de dados que temos.

In [ ]:
# Verificação dos tipos de dados no conjunto de dados de treino

dados.dtypes

Observa-se que no conjunto de dados de treino, existem três tipos de dados distintos:
* Inteiros (int64): são os dados que representam os números digitais expressos nos pixels das imagens, e são diretamente relacionados à resposta espectral dos alvos da cena. Na coluna `id`, representam a identificação do pixel.
* Flutuantes (float64): são dados que representam o resultado de uma estatística que mede a diferença entre uma resposta espectral modelada e a resposta espectral observada;
* Texto (object): são os dados de rótulos dos pixels, representando a verdade terrestre das cenas.

In [ ]:
# Verificação dos tipos de dados no conjunto de dados de treino

dados_validacao.dtypes

O conjunto de dados de teste apresenta a mesma estrutura de tipos de dados que o conjunto de treino. Uma vez que os conjuntos de dados são compatíveis estruturalmente, podemos seguir para a verificação de valores únicos.

In [ ]:
# Valores únicos (dados de treino)

dados.nunique()

In [ ]:
# Valores únicos (dados de teste)

dados_validacao.nunique()

Vamos verificar agora a estatística descritiva do conjunto de dados de treino.

In [ ]:
dados.describe()

# Construção do modelo:

## 1.  Pré-processamento:

In [ ]:
# Remoção da coluna de identificação do conjunto de dados de treino

dados = dados.drop(columns="id")

In [ ]:
# Separação dos atributos e dos rótulos

X = dados.iloc[:,:-1].values # atributos (amostra de pixels)
y = dados.iloc[:,-1].values # rótulos (verdade terreste)

In [ ]:
# Separação das amostras de treino e de teste (80% - treino e 20% - teste)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
# Usamos o random_state = 42 paraeliminar a aleatoriedade caso venhamos a comparar diferentes modelos

# One-hot encoding
encoder = preprocessing.OneHotEncoder(sparse=False)
y_treino = encoder.fit_transform(y_treino.reshape(-1, 1))
y_teste = encoder.fit_transform(y_teste.reshape(-1, 1))
# Informações importantes

print("Formato dos dados de treino:", "X_treino =", X_treino.shape, "e y_treino =", y_treino.shape)
print("Formato dos dados de teste:", "X_teste =", X_teste.shape, "e y_teste =", y_teste.shape)
print()
print("Número de classes:", len(set(dados['label'].values)))
print("Número de features:", X_treino.shape[1])

Formato dos dados de treino: X_treino = (167, 27) e y_treino = (167, 4)
Formato dos dados de teste: X_teste = (42, 27) e y_teste = (42, 4)

Número de classes: 4
Número de features: 27


Um indicativo de que tudo está ok é o fato do número de linhas ser igual para os conjuntos de dados de treino. O conjunto x possui 209 linhas (valores) e 27 colunas (features), enquanto o conjunto y possui 209 linhas (valores) e um única coluna com os rótulos.

Para o conjunto de teste, o indicativo de que está tudo certo é a quantidade de colunas (features), que bate com a quantidade de features do conjunto de treino.

# 3. Criando os modelos

## *Random Forest Classifier*

In [ ]:
rfc = ensemble.RandomForestClassifier(max_features='auto', n_estimators=1000, n_jobs=-1, min_samples_leaf=2, random_state=42, criterion='entropy')

In [ ]:
rfc.fit(X_treino, y_treino)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
rfc.score(X_teste, y_teste)

0.8809523809523809

In [ ]:
predicao_teste = rfc.predict(X_teste)

In [ ]:
precision_score(y_teste, predicao_teste, average="micro")

0.8809523809523809

## Grid Search

In [ ]:
rf4 = ensemble.RandomForestClassifier()

In [ ]:
params = {"max_features": [1,5,10,15,20,25,'auto'], "n_estimators": [1000, 1500], "min_samples_leaf": [1,2,3,4,5], "random_state": [42], "criterion": ["gini", "entropy"]}

In [ ]:
search = GridSearchCV(rf4, params, n_jobs=-1).fit(X_treino, y_treino)

print(f'Melhor resultado: {search.best_score_} para {search.best_params_}')


Melhor resultado: 0.8679144385026738 para {'criterion': 'entropy', 'max_features': 20, 'min_samples_leaf': 3, 'n_estimators': 1500, 'random_state': 42}


In [ ]:
 random_search = RandomizedSearchCV(rf4, params, n_iter=100, n_jobs=-1).fit(X_treino, y_treino)
 print(f'Melhor resultado: {random_search.best_score_} para {random_search.best_params_}')

Melhor resultado: 0.8679144385026738 para {'random_state': 42, 'n_estimators': 1000, 'min_samples_leaf': 2, 'max_features': 25, 'criterion': 'entropy'}


In [ ]:
feature_importances = search.best_estimator_.feature_importances_

In [ ]:
feature_importances

array([0.07653951, 0.21329847, 0.02026893, 0.06863744, 0.00971575,
       0.01818796, 0.00320315, 0.0650577 , 0.04137094, 0.11248468,
       0.12403427, 0.01091852, 0.02623934, 0.02500618, 0.01203808,
       0.00905053, 0.07428282, 0.04265619, 0.00726196, 0.00501521,
       0.00505848, 0.0058373 , 0.0036436 , 0.00314348, 0.00498376,
       0.00321217, 0.00885358])

## Resultado final

In [ ]:
y_verificacao = rfc.predict(dados_val_sid)

In [ ]:
y_verificacao = encoder.inverse_transform(y_verificacao).reshape(-1)

In [ ]:
y_verificacao

array(['h ', 's ', 's ', 'o ', 's ', 'd ', 'h ', 's ', 'h ', 'o ', 's ',
       's ', 's ', 'd ', 'd ', 'o ', 's ', 's ', 'd ', 's ', 'o ', 'd ',
       's ', 's ', 's ', 'd ', 'o ', 's ', 'd ', 's ', 'd ', 's ', 'd ',
       'o ', 'h ', 'h ', 's ', 'd ', 'd ', 'o ', 'h ', 's ', 'd ', 's ',
       'h ', 'o ', 's ', 'o ', 'o ', 'd ', 's ', 'd ', 'h ', 'h ', 'h ',
       's ', 'd ', 'd ', 's ', 's ', 's ', 'h ', 's ', 's ', 'd ', 's ',
       'd ', 'h ', 'd ', 's ', 'd ', 's ', 'd ', 'd ', 'd ', 'h ', 'd ',
       'h ', 's ', 'h ', 'o ', 's ', 's ', 'h ', 'o ', 's ', 'd ', 'd ',
       'd ', 's ', 'h ', 'd ', 's ', 's ', 's ', 's ', 'd ', 'd ', 'o ',
       'h ', 'd ', 'd ', 'h ', 's ', 'h ', 's ', 's ', 's ', 's ', 'd ',
       's ', 'd ', 'o ', 's ', 'd ', 'd ', 'o ', 's ', 'd ', 's ', 'd ',
       's ', 'h ', 's ', 'd ', 'd ', 'h ', 'h ', 's ', 's ', 'o ', 'h ',
       's ', 's ', 'd ', 'h ', 'h ', 'd ', 's ', 'h ', 'd ', 's ', 'd ',
       'd ', 's ', 'o ', 'd ', 's ', 'd ', 'd ', 's

In [ ]:
index = dados_validacao["id"]
columns = ['label']


In [ ]:
output = pd.DataFrame(y_verificacao, index=index, columns=columns)

In [ ]:
output.to_csv('result.csv', sep=",")

In [ ]:

pd.read_csv('result.csv')

,id,label
0,356,h
1,202,s
2,183,s
3,306,o
4,511,s
...,...,...
309,7,d
310,336,d
311,345,d
312,380,s


In [ ]:
from google.colab import files
files.download("result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>